In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
ruta_csv1 = '/content/drive/MyDrive/datasets/dataset_completo.csv'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_movies = pd.read_csv('/content/drive/MyDrive/datasets/movies_dataset.csv')

<ipython-input-3-d1d9a8e438ba>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('/content/drive/MyDrive/datasets/movies_dataset.csv')


In [4]:
df_credits = pd.read_csv('/content/drive/MyDrive/datasets/credits.csv')

In [5]:
df_movies = df_movies[['title', 'overview', 'genres', 'id']]

In [6]:
df_credits['id'] = df_credits['id'].astype(str)

In [7]:
df = pd.merge(df_movies, df_credits, on = 'id', how = 'inner')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45538 entries, 0 to 45537
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     45535 non-null  object
 1   overview  44584 non-null  object
 2   genres    45538 non-null  object
 3   id        45538 non-null  object
 4   cast      45538 non-null  object
 5   crew      45538 non-null  object
dtypes: object(6)
memory usage: 2.4+ MB


In [17]:
# (!)NO EJECUTAR # (!)NO EJECUTAR # (!)NO EJECUTAR # (!)NO EJECUTAR # (!)NO EJECUTAR # (!)NO EJECUTAR
df.to_csv('/content/drive/MyDrive/datasets/modelo_dataset.csv')# (!)NO EJECUTAR # (!)NO EJECUTAR #
# (!)NO EJECUTAR # (!)NO EJECUTAR # (!)NO EJECUTAR # (!)NO EJECUTAR # (!)NO EJECUTAR # (!)NO EJECUTAR

In [9]:
pelicula = 'Toy Story'

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

# Obtén el título de la película de entrada del usuario
nombre_pelicula = pelicula

# Encuentra la película de entrada en el conjunto de datos
pelicula_seleccionada = df[df['title'] == nombre_pelicula]

# Verifica si la película de entrada existe en el conjunto de datos
if len(pelicula_seleccionada) == 0:
    print("La película no se encuentra en el conjunto de datos.")
else:
    # Obtén las características relevantes de las películas
    features = ['title', 'overview', 'genres', 'cast', 'crew']

    # Crea un nuevo DataFrame solo con las características relevantes
    df_selected = df[features]

    # Elimina las filas con valores faltantes
    df_selected = df_selected.dropna()

    # Combina las columnas 'cast' y 'crew' en una nueva columna llamada 'cast_crew'
    df_selected['cast_crew'] = df_selected['cast'] + ' ' + df_selected['crew']

    # Crea una matriz TF-IDF para representar los textos de las películas y elenco/equipo
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_selected['overview'] + ' ' + df_selected['cast_crew'])

    # Definir el tamaño del lote y calcular el número de lotes
batch_size = 5000  # Tamaño del lote para el cálculo de similitud
num_movies = len(df_selected)
num_batches = (num_movies // batch_size) + 1

cosine_similarities = None

# Iterar a través de los lotes de películas
for batch_index in range(num_batches):
    start_index = batch_index * batch_size
    end_index = min(start_index + batch_size, num_movies)

    # Obtener el subconjunto de películas para el lote actual
    movies_subset = range(start_index, end_index)

    # Crear una matriz TF-IDF solo con las películas del lote actual
    tfidf_matrix_subset = tfidf_matrix[movies_subset]

    # Calcular la similitud coseno entre las películas del lote actual
    batch_cosine_similarities = linear_kernel(tfidf_matrix_subset, tfidf_matrix_subset)

    if cosine_similarities is None:
        cosine_similarities = batch_cosine_similarities
    else:
        # Ajustar la dimensión del último lote si es necesario
        if batch_cosine_similarities.shape[0] < batch_size:
            batch_cosine_similarities = np.pad(batch_cosine_similarities, ((0, batch_size - batch_cosine_similarities.shape[0]), (0, 0)), mode='constant')
        elif batch_cosine_similarities.shape[0] > batch_size:
            batch_cosine_similarities = batch_cosine_similarities[:batch_size]

        # Ajustar la dimensión de cosine_similarities si es necesario
        if cosine_similarities.shape[1] < batch_cosine_similarities.shape[1]:
            cosine_similarities = np.pad(cosine_similarities, ((0, 0), (0, batch_cosine_similarities.shape[1] - cosine_similarities.shape[1])), mode='constant')
        elif cosine_similarities.shape[1] > batch_cosine_similarities.shape[1]:
            batch_cosine_similarities = np.pad(batch_cosine_similarities, ((0, 0), (0, cosine_similarities.shape[1] - batch_cosine_similarities.shape[1])), mode='constant')

        # Combinar los resultados del lote actual con los resultados anteriores
        cosine_similarities = np.concatenate((cosine_similarities, batch_cosine_similarities), axis=0)


# ...

# Obtén el índice de la película seleccionada
idx = df_selected[df_selected['title'] == nombre_pelicula].index[0]

# Obtén las puntuaciones de similitud de la película seleccionada con todas las demás películas
similarity_scores = list(enumerate(cosine_similarities[idx]))

# Ordena las películas según su similitud
similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

# Muestra las 5 películas más similares
similar_movies = similarity_scores[1:6]  # Excluye la película de entrada
print("Películas similares a", nombre_pelicula, ":")
for movie in similar_movies:
    similar_movie_title = df_selected.iloc[movie[0]]['title']
    print(similar_movie_title)

<ipython-input-13-fa3eccd3be52>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['cast_crew'] = df_selected['cast'] + ' ' + df_selected['crew']


Películas similares a Toy Story :
Toy Story 2
Monsters, Inc.
Snow White and the Seven Dwarfs
South Park: Bigger, Longer & Uncut
James and the Giant Peach
